#### This is a [jupyter](https://jupyter.org/) notebook that contains text, equations, images and executable code in one document. It makes use of  [FEniCSx](https://fenicsproject.org/), an open-source framework supporting Finite Element Methods.

In [1]:
import pyvista as pv

# Finite Element Method for sound fields
## 1 Partition of the domain

The first step of the FEM is the partition of the domain into simply shaped sub-domains. These are called the **Finite Elements**. The reason for this will become evident later on, when we will learn how to perform the integration over the domain. The integral is more easy to solve if we split it into a sum of integrals over the elements.

\begin{equation}
\int_\Omega ... dV = \sum_i \int_{\Omega_i} ... dV
\end{equation}

The process of partitioning will generate a **mesh**. A mesh is a collection of nodes end elements. Each element is assigned some nodes, while each node is assigned to one or more elements.

### Elements

In acoustics, the domain may be one-, two-, or three-dimensional. Depending on the dimensionality, different element shapes may be used.

For one-dimensional domains (representing pipes and similar) the elements are lines. Here are some examples of such line elements (straight or curved) with two, three and four nodes:

In [2]:
from pyvista import CellType as ct
style = {'show_vertices':True, 'cpos':'xy', 'point_size':6, 'render_points_as_spheres':True, 'line_width':3}
shapes = [ct.LINE, ct.QUADRATIC_EDGE, ct.CUBIC_LINE]
points = [[-1,1,0],[1,1,0],[-1,0.5,0],[1,0.5,0],[0,0.5,0],[-1,0,0],[1,0,0],[-0.33,0,0],[.33,-0,0]]
elements = [[2,0,1],[3,2,3,4],[4,5,6,7,8]]
pv.UnstructuredGrid(sum(elements,[]), shapes, points).plot(**style)

Widget(value='<iframe src="http://localhost:46739/index.html?ui=P_0x7aa2494d4050_0&reconnect=auto" class="pyvi…

For two-dimensional domains, the elements are generally polygons. It is common to use either triangles or quadrilaterals. Here is an example with triangle elements with three and with six nodes and quadrilateral with four nodes:

In [3]:
shapes = [ct.TRIANGLE, ct.QUADRATIC_TRIANGLE, ct.QUAD]
points = [[0,0,0],[1,0,0],[0.5,0.707,0],[2,0,0],[3,0,0],[2.5,0.707,0],[2.5,0,0],[2.75,0.353,0],[2.25,0.353,0],
          [4,0,0],[4.7,0,0],[4.7,0.7,0],[4,0.7,0]]
elements = [[3,0,1,2],[6,3,4,5,6,7,8],[4,9,10,11,12]]
pv.UnstructuredGrid(sum(elements,[]), shapes, points).plot(**style)

Widget(value='<iframe src="http://localhost:46739/index.html?ui=P_0x7aa1bbc90e10_1&reconnect=auto" class="pyvi…

Finally, in case of three-dimensional domains, the elements are polyhedrons. Tetrahedron, hexahedron, pyramids and wedges are the most common shapes. The example shows a tretrahedron with four nodes and a hexahedron with eight nodes.

In [4]:
shapes = [ct.TETRA, ct.HEXAHEDRON]
points = [[0,0,0],[1,0,0],[0.5,0.707,0],[0.5,0.35,0.577],[2,0,0],[2.7,0,0],[2.7,0.7,0],[2,0.7,0],[2,0,0.7],[2.7,0,0.7],[2.7,0.7,0.7],[2,0.7,0.7]]
elements = [[4,0,1,2,3],[8,4,5,6,7,8,9,10,11]]
pv.UnstructuredGrid(sum(elements,[]), shapes, points).plot(**style)

Widget(value='<iframe src="http://localhost:46739/index.html?ui=P_0x7aa249587890_2&reconnect=auto" class="pyvi…

### Meshes
The partition of a given domain into finite elements can be achieved in many ways. In practice we will rely on some algorithm doing the work. Depending on the kind of algorithm the outcome can be controlled using parameters, such as the minimum or maximum size of the elements or the number of elements.

The following example of a 2D domain shaped like a cross-section of a car interior will demonstrate this. It becomes also obvious that the generic triangle shape of the elements needs to get distorted in order to produce the mesh. The algorithm tries to produce a mesh where the elements are not too different in size and have edges that are not too different in length.

In [5]:
from mpi4py import MPI
import gmsh
from dolfinx.io import gmshio
from dolfinx import plot
# first we set up the 'solid model' which describes the geometry 
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 0)
model = gmsh.model()
model.add("car")
lc = 4
coords = ((0.,0.),(3.,0.),(3.3,1),(4.5,1),(3.5,2),(1.,2),(-.5,1.2))
for x,y in coords:
    n = model.geo.addPoint(x, y, 0, lc)
for i in range(n):
    model.geo.addLine(i+1,(i+1)%n+1)
model.geo.addCurveLoop(range(1,n+1),1)
model.geo.addPlaneSurface([1],1)
model.geo.synchronize()
model.addPhysicalGroup(2, [1], name="My surface")
pl = pv.Plotter(shape=(2,2))
# now the geometry is 'meshed'
# the example show four different variants of meshes
mesh_sizes = (3,1.8,0.6,0.2)
for i,m in enumerate(mesh_sizes):
    gmsh.option.setNumber("Mesh.MeshSizeMax",m)
    model.mesh.generate(dim=1)
    model.mesh.generate(dim=2)
    car_mesh,_,_ = gmshio.model_to_mesh(model,MPI.COMM_SELF,0,gdim=2)
    grid = pv.UnstructuredGrid(*plot.vtk_mesh(car_mesh, 2))
    pl.subplot(i//2,i%2)
    pl.add_mesh(grid,**{'show_edges':True, 'show_vertices':False})
    pl.view_xy()
pl.show()

Widget(value='<iframe src="http://localhost:46739/index.html?ui=P_0x7aa152b3bc50_3&reconnect=auto" class="pyvi…

Complicated three dimensional shapes are most easily meshed using tetrahedrons. The examples show a car interior shape in 3D and a cutout of a sphere from a cube.

In [6]:
gmsh.model.geo.extrude([[2,1]], 0, 0, 1.5)
model.geo.synchronize()
model.addPhysicalGroup(3, [1], name="Car interior 3D")
gmsh.option.setNumber("Mesh.MeshSizeMax",0.5)
model.mesh.generate(dim=3)
mesh,_,_ = gmshio.model_to_mesh(model,MPI.COMM_SELF,0,gdim=3)
gmsh.finalize()
grid = pv.UnstructuredGrid(*plot.vtk_mesh(mesh, 3))
xx = grid.plot(**{'show_vertices':False, 'show_edges':True, 'cpos': [(-6, 6, 7), (2.0, 1.0, 0.75), (0.0, 1.0, 0.0)]})

Widget(value='<iframe src="http://localhost:46739/index.html?ui=P_0x7aa152b3bed0_4&reconnect=auto" class="pyvi…

In [7]:
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 0)
model = gmsh.model()
model.add("CubeSphere")
sphere = model.occ.addSphere(0.5, 0.5, 0.6, 0.55, tag=1)
cube =model.occ.addBox(0,0,0,1,1,1, tag=2)
vol = gmsh.model.occ.cut([(3, cube)], [(3, sphere)])
model.occ.synchronize()
volumes = gmsh.model.getEntities(dim=3)
model.addPhysicalGroup(3, [2], name="Volume")
gmsh.option.setNumber("Mesh.Algorithm", 1)
gmsh.option.setNumber("Mesh.MeshSizeMax",0.1)
model.mesh.generate(dim=3)
mesh,_,_ = gmshio.model_to_mesh(model,MPI.COMM_SELF,0,gdim=3)
gmsh.finalize()
grid = pv.UnstructuredGrid(*plot.vtk_mesh(mesh, 3))
grid.plot(**{'show_vertices':False, 'show_edges':True, 'cpos': 'iso'})

Widget(value='<iframe src="http://localhost:46739/index.html?ui=P_0x7aa1529cc190_5&reconnect=auto" class="pyvi…

## 2 Interpolation of field quantities inside the finite elements

The second step in the FEM is the interpolation of the field quantity (e.g. sound pressure) inside each of the finite elements. This can be achieved using a general interpolation

\begin{equation}
p(\mathbf{x})=\sum_i N_i(\mathbf{x}) p_i=\mathbf{N}^\mathrm{T}\mathbf{p}
\end{equation}

where the supporting points $i$ are the nodes of the finite element (see examples). The interpolating functions are the **shape functions** $N_i$. The choice of these shape functions is important to provide some mathematical properties to the interpolation that are of advantage later on, when the interpolation is used as part of integrands. The values of the field quantity (or generally field quantities) $p_i$ at the nodes are the **degrees of freedom** (DOF). In acoustics it is often sufficient to have just one DOF - the sound pressure $p$.

## 3 Integral formulation
To complete the FEM, we need a formulation of the generic problem (PDE, BC) in a form with only integrals. These have then to be computed. This can be achieved by either using
- a variational approach to formulate the integrals (the PDE is needed only implicitly)
- the transformation of the PDE into definite integrals over the elements

We will have a look into the details later on. To wet you appetite, we use the car interior from above and compute the sound field produced by a source at a frequency of 200 Hz.

In [8]:
import numpy as np
import ufl
from dolfinx import fem
from dolfinx.fem.petsc import LinearProblem
om = 210.*2*np.pi
c = 343.
# function space and functions
V = fem.functionspace(car_mesh,("Lagrange", 1))
k0 = fem.Constant(car_mesh,om/c)
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
# source
Sx, Sy = 1.0,1.0
alfa = 0.015
delta_tmp = fem.Function(V)
delta_tmp.interpolate(lambda x : np.exp(-(((x[0]-Sx)**2+(x[1]-Sy)**2)/(alfa**2))))
int_delta_tmp = fem.assemble_scalar(fem.form(delta_tmp*ufl.dx)) 
f = delta_tmp/int_delta_tmp
# weak form
a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx - k0**2 * ufl.inner(u, v) * ufl.dx
L = ufl.inner(f, v) * ufl.dx
# solution
p = LinearProblem(a, L).solve()
# visualization
grid = pv.UnstructuredGrid(*plot.vtk_mesh(car_mesh, 2))
grid.point_data['p'] = p.x.array
pl = pv.Plotter()
pl.add_mesh(grid, show_edges=True, edge_color='gray')
pl.add_mesh(pv.Sphere(0.05,(1,1,0)),color='red')
pl.view_xy()
pl.show()

Widget(value='<iframe src="http://localhost:46739/index.html?ui=P_0x7aa1529cc7d0_6&reconnect=auto" class="pyvi…

#### License

This notebook is an [Open Educational Resource](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use it for your own purposes. The text and the images are licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), and any code under the [MIT license](https://opensource.org/licenses/MIT). Please attribute the work as follows: Ennes Sarradj, Numerical Acoustics: Finite Element Method for sound fields, 2025.